In [1]:
import sys

import multiprocess as mp  # NOT multiprocessing from the default library -- we need dill
import numpy as np
import pandas as pd
import xarray as xr
from statsmodels.formula.api import ols


In [2]:
cov = [[1.0, 0.5], [0.5, 1.0]]  # Covariance between y0 and y1
size = 30  # total cell size, after data "peeking"
cond_list = [-1, 0, 1]  # Three level coding of condition variable

# The rhs of the fit formulas mapped to list of p-values will be inspected.
fit_data = {
    "cond": ["cond"],
    "cond + sex": ["cond"],
    "cond + sex + cond*sex": ["cond", "cond:sex"],
}

# Values for each "researcher d.o.f.""
coords=(
    ("y", ["y0", "y1", "ya"]),  # either correlated variable, or their average
    ("count", [20, 30]),  # baseline is 20, but 30 is used if 20 is insignificant
    ("rhs", list(fit_data.keys())),  # fits with or without main & interaction effects
    ("cond", cond_list + [None]),  # which conditions to throw out
)

# Selection of pvalues that get inspected in each scenario.
sel = {
    "A": dict(count=20, cond=-1, rhs="cond"),  # Any dependent variable
    "B": dict(y="y0", cond=-1, rhs="cond"),  # Optionally use a larger sample.
    "C": dict(y="y0", count=20, cond=-1),  # Inspect multiple fit formulas.
    "D": dict(y="y0", count=20, rhs="cond"),  # Try different combinations of conditions.
    # Researchers also may accidentally use multiple of the above:
    "AB": dict(cond=-1, rhs="cond"),
    "ABC": dict(cond=-1),
    "ABD": dict(rhs="cond"),  # Original paper incorrectly showed ABD for ABC.
    "ABCD": dict()
}


In [3]:
def run_sim(seed):
    """Run one data simulation and calculate pvalue array."""
    df = make_df(seed)
    return calc_pvals(df)


def make_df(seed):
    """Create a simulated dataset.
    
    Parameters
    ----------
    seed: seed for random number generator
    
    Returns
    -------
    pd.DataFrame with columns:
          cond: experimental condition
          n: experimental unit index within a condition
          y0 and y1: variable random draw from 2d multivariate normal
          ya: average of y0 and y1
          sex: randomly assigned sex for each unit
    """
    rng = np.random.default_rng(seed)
    
    def make_one():
        """Make a dataframe for one condition."""
        data = rng.multivariate_normal([0, 0], cov, size)
        sex = rng.binomial(1, 0.5, size)
        return pd.DataFrame({"y0": data[:, 0], "y1": data[:, 1], "sex": sex})
    
    # Combine all dataframes set up "cond" and "n" columns
    df = pd.concat([make_one() for _ in cond_list], keys=cond_list, names=["cond", "n"])
    
    df["ya"] = (df["y0"] + df["y1"]) / 2
    return df.reset_index()


def calc_pvals(df):
    """Calculate pvalues for all possible datasets and fits that may be used.
    
    Parameters
    ----------
    df: pd.DataFrame output by `make_df` above
    
    Returns
    -------
    xr.DataArray with dimensions:
        y: y variable fitted
        count: dataset size
        rhs: fit formula
        cond: which condition was excluded
    """
    pvals = xr.DataArray(coords=coords)

    for count in pvals["count"].values:
        df_count = df[df.n < count]
        for cond in pvals["cond"].values:
            df_fit = df_count[df_count.cond != cond]
            for y in pvals["y"].values:
                for rhs in pvals["rhs"].values:
                    fit = ols(f"{y} ~ {rhs}", data=df_fit).fit()
                    
                    # Take minimum across all indicated p-values.  Multiple
                    # p-values are inspected when fitting with an interaction.
                    p_labels = fit_data[rhs]
                    p_min = min(fit.pvalues[p_labels])
                    
                    pvals.loc[y, count, rhs, cond] = p_min
                    
    return pvals


In [4]:
# Run the simulations, each with a different seed.
num_sim = 15_000
rng_init = np.random.default_rng(0)
seeds = rng_init.integers(sys.maxsize, size=num_sim)

pool = mp.Pool()
arr_list = pool.map(run_sim, seeds)  # multi-process
#arr_list = list(map(run_sim, seeds))  # single-threaded


In [5]:
# Concatenate all simulation results and find lowest p-value for the
# criteria in a given analysis scenario.
arr = xr.concat(arr_list, dim="sim")
pvals = {}
for scenario, criteria in sel.items():
    arr_sel = arr.loc[criteria]
    # Take minimum across all remaining dimensions except the simulation index
    pvals[scenario] = arr_sel.min(d for d in arr_sel.dims if d != "sim")
df_pvals = pd.DataFrame(pvals)

# Find the number of allegedly significant simulations within each scenario.
results = []
for level in 0.1, 0.05, 0.01:
    sig = (df_pvals < level).mean()
    sig = {"significance level": level, **sig.to_dict()}
    results.append(sig)
df_results = pd.DataFrame(results).set_index("significance level").T
df_results.index.name = "researcher d.o.f."

# Print output table
df_results


significance level,0.10,0.05,0.01
researcher d.o.f.,,,
A,0.181133,0.095733,0.021200
B,0.146333,0.075467,0.016467
C,0.216800,0.117067,0.026667
D,0.227133,0.123933,0.027933
AB,0.266267,0.146933,0.034733
ABC,0.509133,0.312533,0.087667
ABD,0.527400,0.332867,0.093200
ABCD,0.817533,0.610533,0.217867
